In [33]:
import pathlib 
import os
import pandas as pd  
import random

In [2]:
root_dir = '../../../data/data/Desc_dataset/'
style_enc_file = '../../../data/data/Desc_dataset/style_enc.csv'

## rename files by giving style index

In [3]:
root_path = pathlib.Path(root_dir)
idxs = []
styles = []
style_enc = {}
n = 1
m = 0
for file in root_path.glob('*'):
    m += 1
    st_name = ''.join(file.stem.split("_")[:-1])
    ind = file.stem.split("_")[-1]
    if st_name not in style_enc:
        style_enc[st_name] = n
        n+=1
    os.rename(file, os.path.join(file.parent, f'{m}.jpg'))
    idxs.append(m)
    styles.append(style_enc[st_name])
st_df = pd.DataFrame(data={'fname' : idxs, 'style_code' : styles})
st_df.to_csv(style_enc_file, index=False)

## build tf dataset and datapipeline

In [4]:
import tensorflow as tf  

In [31]:
root_path = pathlib.Path(root_dir)
list_ds = tf.data.Dataset.list_files(str(root_path/'*.jpg'))
stenc_df = pd.read_csv(style_enc_file)['style_code'].tolist()
LEN = len(stenc_df)

In [32]:
LEN 

8355

In [6]:
list_ds.element_spec

TensorSpec(shape=(), dtype=tf.string, name=None)

In [7]:
for f in list_ds.take(5):
    print(f.numpy())

b'..\\..\\..\\data\\data\\Desc_dataset\\7502.jpg'
b'..\\..\\..\\data\\data\\Desc_dataset\\2139.jpg'
b'..\\..\\..\\data\\data\\Desc_dataset\\6467.jpg'
b'..\\..\\..\\data\\data\\Desc_dataset\\2393.jpg'
b'..\\..\\..\\data\\data\\Desc_dataset\\5196.jpg'


In [55]:
def process_path(file_path):
    cur_ind = tf.strings.split(tf.strings.split(file_path, os.sep)[-1],'.')[0]
    # cur_style = stenc_df[cur_ind-1]
    # random_num = random.randint(0, LEN)
    # if random_num == int(cur_ind):
    #     random_num = random.randint(0, LEN)
    # sec_style = stenc_df[random_num-1]
    # cur_img = tf.io.read_file(file_path)
    #rand_path = tf.strings.join(inputs=tf.strings.split(file_path, os.sep)[:-1], separator=os.sep)
    #rand_img = tf.io.read_file(tf.strings.join(inputs=[*tf.strings.split(file_path, os.sep)[:-1],os.sep,f"{random_num}.jpg"], seperator=os.sep))
    #cur_ind = tf.strings.split(file_path, os.sep)[-1]
    return type(cur_ind)

In [48]:
def process_path(file_path):
  label = tf.strings.split(file_path, os.sep)[-2]
  return tf.io.read_file(file_path)

In [56]:
sample_dt = list_ds.map(process_path)
for f in sample_dt.take(5):
    print(f.numpy())

TypeError: Unsupported return value from function passed to Dataset.map(): <class 'tensorflow.python.framework.ops.Tensor'>.